In [1]:
!pip install tensorflow==1.13.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 42.1 MB/s eta 0:00:0000:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.6/367.6 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.4
    Uninstalling tensorflow-2.6.4:
      Successfully uninstalled tensorflow-2.6.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [2]:
import tensorflow as tf
import os
import time
import pickle
import numpy as np

print(tf.__version__)
CIFAR_DIR = '/kaggle/input/cifar10/cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1
['data_batch_1', 'data_batch_2', 'batches.meta', 'test_batch', 'data_batch_3', 'data_batch_5', 'data_batch_4', 'readme.html']


In [3]:
train_filenames = [os.path.join(CIFAR_DIR, f'data_batch_{i}') for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]
print(train_filenames)
print(test_filenames)

['/kaggle/input/cifar10/cifar-10-batches-py/data_batch_1', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_2', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_3', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_4', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_5']
['/kaggle/input/cifar10/cifar-10-batches-py/test_batch']


In [4]:
def load_data(filename):
    """前面是图片特征，后面是图片分类标签"""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']


# tensorflow.Dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_lables = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)  # all_data[0]列表中10000个存有(32*32*3=)3072个数的一维列表c
            all_lables.append(labels)  # all_data[0]列表中10000个标签值
        self._data = np.vstack(all_data)  # 垂直方向沿轴2堆叠 变为ndarray
        self._data = self._data / 127.5 - 1  # 归一化到范围[-1,1]
        self._lables = np.hstack(all_lables)  # 一维列表变成ndarray
        print(self._data.shape)  # (50000, 3072) 每一张图像展平后的结果
        print(self._lables.shape)
        print(self._lables[:5])

        self._num_samples = self._data.shape[0]  # 样本数
        self._need_shuffle = need_shuffle  # 进行洗牌随机
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()

    def _shuffle_data(self):
        # 通过随机排列下标进行随机样本
        p = np.random.permutation(self._num_samples)  # 变成随机下标
        self._data = self._data[p]
        self._lables = self._lables[p]

    def next_batch(self, batch_size):
        """return batch_size samples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_samples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = self._indicator + batch_size
            else:
                raise Exception('have no more samples')
        if end_indicator > self._num_samples:
            # 去除不足batch_size剩余的样本
            raise Exception('batch size is larger than the remaining samples')
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._lables[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels


train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
[6 9 9 4 1]
(10000, 3072)
(10000,)
[3 8 8 0 6]


In [5]:
# 构造分组卷积块
def inception_block(x, output_channel_for_each_path,  # 每一个分组卷积的输出通道数
                    name):
    """inception block implementation"""
    with tf.variable_scope(name):
        conv1_1 = tf.layers.conv2d(x,
                                   output_channel_for_each_path[0],
                                   (1, 1),
                                   strides=(1, 1),
                                   padding='same',
                                   activation=tf.nn.relu,
                                   name='conv1_1')
        conv3_3 = tf.layers.conv2d(x,
                                   output_channel_for_each_path[1],
                                   (3, 3),
                                   strides=(1, 1),
                                   padding='same',
                                   activation=tf.nn.relu,
                                   name='conv3_3')
        conv5_5 = tf.layers.conv2d(x,
                                   output_channel_for_each_path[2],
                                   (1, 1),
                                   strides=(1, 1),
                                   padding='same',
                                   activation=tf.nn.relu,
                                   name='conv5_5')
        max_pooling = tf.layers.max_pooling2d(x,
                                              (2, 2),
                                              (2, 2),
                                              name='max_pooling')
    max_pooling_shape = max_pooling.get_shape().as_list()[1:]
    input_shape = x.get_shape().as_list()[1:]
    height_padding = (input_shape[0] - max_pooling_shape[0]) // 2
    wideth_padding = (input_shape[1] - max_pooling_shape[1]) // 2
    padding_pooling = tf.pad(max_pooling, [[0, 0], [height_padding, height_padding],
                                           [wideth_padding, wideth_padding], [0, 0]])
    # 在通道数维度进行合并
    concat_layer = tf.concat([conv1_1, conv3_3, conv5_5, padding_pooling], axis=3)
    return concat_layer

In [6]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
x_image = tf.reshape(x, [-1, 3, 32, 32])
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

In [7]:
# 搭建InceptionNet
conv1 = tf.layers.conv2d(x_image,
                         32,
                         (3, 3),
                         padding='same',
                         activation=tf.nn.relu,
                         name='conv1')
pooling1 = tf.layers.max_pooling2d(conv1,
                                   (2, 2),
                                   (2, 2),
                                   name='pool1')
inception_2a = inception_block(pooling1,
                               [16, 16, 16],
                               name='inception_2a')
inception_2b = inception_block(pooling1,
                               [16, 16, 16],
                               name='inception_2b')
pooling2 = tf.layers.max_pooling2d(inception_2b,
                                   (2, 2),
                                   (2, 2),
                                   name='pool2')
inception_3a = inception_block(pooling2,
                               [16, 16, 16],
                               name='inception_3a')
inception_3b = inception_block(inception_3a,
                               [16, 16, 16],
                               name='inception_3b')
pooling3 = tf.layers.max_pooling2d(inception_3b,
                                   (2, 2),
                                   (2, 2),
                                   name='pool3')
flatten = tf.layers.flatten(pooling3)
print(flatten)
y_ = tf.layers.dense(flatten, 10)
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)  # y是真实值
# 计算张量维度上元素的平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
with tf.name_scope('train_op'):
    # 这里计算梯度并更新了梯度
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Tensor("flatten/Reshape:0", shape=(?, 2816), dtype=float32)


In [8]:

init = tf.global_variables_initializer()  # 低版本tf必须要做
batch_size = 20
train_steps = 10000
test_steps = 100
with tf.Session() as s:
    s.run(init)  # 低版本tf必须要做
    start = time.time()
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = s.run([loss, accuracy, train_op],
                                      feed_dict={x: batch_data, y: batch_labels})
        if (i + 1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, accuracy: %4.5f'
                  % (i + 1, loss_val, accu_val))
            end = time.time()
            print(f'just 100 steps take time: {end - start}s')
            start=end
        if (i + 1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)  # 重新拿测试集
            all_test_accu_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_accu_val = s.run([accuracy],
                                      feed_dict={x: test_batch_data, y: test_batch_labels})
                all_test_accu_val.append(test_accu_val)
            test_accu = np.mean(all_test_accu_val)
            print('[Test ] Step: %d, accuracy: %4.5f' % (i + 1, test_accu))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


2022-07-29 03:36:43.267255: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-07-29 03:36:43.273146: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000170000 Hz
2022-07-29 03:36:43.273615: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x562f9cb34c40 executing computations on platform Host. Devices:
2022-07-29 03:36:43.273642: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>


[Train] Step: 100, loss: 1.89746, accuracy: 0.25000
just 100 steps take time: 5.194921493530273s
[Train] Step: 200, loss: 1.60905, accuracy: 0.45000
just 100 steps take time: 5.525650262832642s
[Train] Step: 300, loss: 1.61515, accuracy: 0.30000
just 100 steps take time: 6.0603015422821045s
[Train] Step: 400, loss: 1.80378, accuracy: 0.25000
just 100 steps take time: 6.656144857406616s
[Train] Step: 500, loss: 1.72245, accuracy: 0.50000
just 100 steps take time: 5.250305414199829s
[Train] Step: 600, loss: 1.70323, accuracy: 0.35000
just 100 steps take time: 5.181459426879883s
[Train] Step: 700, loss: 1.63959, accuracy: 0.25000
just 100 steps take time: 4.916364908218384s
[Train] Step: 800, loss: 1.57610, accuracy: 0.40000
just 100 steps take time: 5.239777565002441s
[Train] Step: 900, loss: 1.30174, accuracy: 0.60000
just 100 steps take time: 5.028223037719727s
[Train] Step: 1000, loss: 1.22131, accuracy: 0.55000
just 100 steps take time: 5.813836574554443s
(10000, 3072)
(10000,)
[3 8 